# List the files in the directory

In [ ]:
!ls /Dataset-IK/

ls: cannot access '/Dataset-IK/': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls /content/drive/MyDrive/IK/Dataset-IK/

ls: cannot access '/content/drive/MyDrive/IK/Dataset-IK/': No such file or directory


In [ ]:
# The PDF files for each author are stored in their respective folder
import os
len(os.listdir('/content/drive/MyDrive/IK/Dataset-IK/')) #No. of authors

20

In [ ]:
c = 0 # No. of files
for root, dirs, files in os.walk('/content/drive/MyDrive/IK/Dataset-IK/'):
    for file in files:
        c = c + 1
        #print(os.path.join(root, file))

print('Total No. of PDF files (research papers):',c)

Total No. of PDF files (research papers): 112


# Extracting text from PDFs

In [ ]:
!pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=a59c5cb3df317437daa639b7210e3a4b7365a4c2f5c00e7673c811807817eb8a
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
import tika #PDF Text Extractor
from tika import parser

# Set up tika
tika.initVM()

# Function to extract text from PDF
def extract_text(filename):
    # Parse PDF file
    parsed = parser.from_file(filename)

    # Extract text from parsed data
    text = parsed["content"]

    return text

In [ ]:
#Testing on a sample PDF file
filename = '/content/drive/MyDrive/IK/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf'
extract_text(filename)

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTitle:\n\n\nA Review of Clustering Techniques and Developments \n\nAmit Saxena1, Mukesh Prasad2, Akshansh Gupta3, Neha Bharill4, Om Prakash Patel4, Aruna Tiwari4, \n\nMeng Joo Er5, Weiping Ding6, Chin-Teng Lin2  \n\n1Department of Computer Science & IT, Guru Ghasidas Vishwavidyalaya, Bilaspur, India \n\n2Centre for Artificial Intelligence, University of Technology Sydney, Sydney, Australia \n\n3School of Computational and Integrative Sciences, Jawaharlal Nehru University, New Delhi, India \n\n4Department of Computer Science and Engineering, Indian Institute of Technology Indore, India \n\n5School of Electrical and Electronic Engineering, Nanyang Technological University, Singapore \n\n6School of Computer and Technology, Nantong University, Nantong, China \n\n \n\n \n\nAbstract \n\nThis paper presents a comprehensive study on clustering: exiting methods and developments made at various \n\ntimes. Clustering

## If you only want to extract the 'Abstract' from a research paper
### Please note that this function might not work correctly for some PDFs.  

In [ ]:
# Function to extract abstract from PDF
def extract_abstract(filename):
    # Parse PDF file
    parsed = parser.from_file(filename)
    print(type(parsed))

    # Extract text from parsed data
    text = parsed["content"]
    # Find introduction section by looking for keywords
    abstract_start = text.find("Abstract")
    print(f"abstract_start {abstract_start}")
    abstract_end_options = [text.find("\n1 ", abstract_start), text.find("\ni ", abstract_start), text.find("\nI ", abstract_start)]
    abstract_end = min(pos for pos in abstract_end_options if pos != -1)
    # Extract introduction section
    abstract = text[abstract_start:abstract_end]

    return abstract

In [ ]:
filename = '/content/drive/MyDrive/IK/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf'
extract_abstract(filename)

<class 'dict'>
abstract_start 794


'Abstract \n\nThis paper presents a comprehensive study on clustering: exiting methods and developments made at various \n\ntimes. Clustering is defined as an unsupervised learning where the objects are grouped on the basis of some \n\nsimilarity inherent among them. There are different methods for clustering the objects such as hierarchical, \n\npartitional, grid, density based and model based. The approaches used in these methods are discussed with their \n\nrespective states of art and applicability. The measures of similarity as well as the evaluation criteria, which are \n\nthe central components of clustering are also presented in the paper. The applications of clustering in some \n\nfields like image segmentation, object and character recognition and data mining are highlighted. \n\nKeywords: Unsupervised learning, Clustering, Data mining, Pattern recognition, Similarity measures \n'

# Extracting Introduction

In [ ]:
def extract_intro(filename):
    # Parse PDF file
    parsed = parser.from_file(filename)

    # Extract text from parsed data
    text = parsed["content"]

    # Find introduction section by looking for the "1 Introduction" heading
    intro_start = text.find("Introduction")

    # Find the next heading that starts with "2 ", "ii ", or "II "
    intro_end_options = [text.find("\n2 ", intro_start), text.find("\nii ", intro_start), text.find("\nII ", intro_start)]
    intro_end = min(pos for pos in intro_end_options if pos != -1)

    # Extract the introduction section
    intro = text[intro_start:intro_end]

    return intro

# Using NER to extract author names

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python3 -m spacy download en_core_web_sm

### Via 'spacy'
spaCy is a free open-source library for Natural Language Processing in Python. It features NER, POS tagging, dependency parsing, word vectors and more.

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
import PyPDF2 # Another text extractor. You can use it when tika doesn't work.
import spacy

def extract_author_names(filename):

    # Load the spaCy NER model
    nlp = spacy.load('en_core_web_sm')

    # Open the PDF file
    pdf_file = open(filename, 'rb')

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Extract the text from the first page
    first_page = pdf_reader.pages[0]
    first_page_text = first_page.extract_text()

    # Close the PDF file
    pdf_file.close()

    # Apply the spaCy NER model to the text
    doc = nlp(first_page_text)
    author_names = []
    for entity in doc.ents:
        if entity.label_ == 'PERSON':
            author_names.append(entity.text)

    # Return the list of author names
    return author_names

In [ ]:
filename

'/content/drive/MyDrive/IK/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf'

In [ ]:
extract_author_names(filename)

['Aruna Tiwari4',
 'Meng Joo',
 'Weiping Ding6',
 'Chin -Teng Lin2',
 'Guru Ghasidas Vishwavidyal']

## Extract 'Abstract', 'Introduction', and 'Author_Names' together

In [ ]:
import spacy
!python3 -m spacy download en_core_web_sm

In [ ]:
!python3 -m spacy download en_core_web_lg

In [ ]:
!python3 -m spacy download en

In [ ]:
import os
import tika
from tika import parser

#tqdm --> Can help you with the progress bar

# Set up Tika
tika.initVM()

#Testing on a sample PDF file
filename = '/content/drive/MyDrive/IK/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf'

# Extract the introduction from the PDF file
text = extract_text(filename)
intro = extract_intro(filename)
abstract = extract_abstract(filename)
author_names = extract_author_names(filename)
print(author_names)

<class 'dict'>
abstract_start 794
['Aruna Tiwari4', 'Meng Joo', 'Weiping Ding6', 'Chin -Teng Lin2', 'Guru Ghasidas Vishwavidyal']


# Complete Extraction

In [ ]:
import os
import tika
from tika import parser

#tqdm --> Can help you with the progress bar

# Set up Tika
tika.initVM()

folder_names = []
texts = []
filenames = []
introduction_vector = []
abstract_vector = []
author_names_vector = []

c = 0 # For printing indexes of the PDF files on which tika couldn't extract the text successfully

for root, dirs, files in os.walk('/content/drive/MyDrive/IK/Dataset-IK/'):
     for file in files:
        #print(os.path.join(root, file))

        c = c + 1

        try:
          # /content/drive/MyDrive/IK/Dataset-IK/AUTHOR_NAME/PAPER_NAME
            # Extract the introduction from the PDF file
            text = extract_text(os.path.join(root, file))
            intro = extract_intro(os.path.join(root, file))
            abstract = extract_abstract(os.path.join(root, file))
            author_names = extract_author_names(os.path.join(root, file))

            folder_names.append(root.split('/')[-1])
            texts.append(text)
            introduction_vector.append(intro)
            abstract_vector.append(abstract)
            author_names_vector.append(author_names)
            filenames.append(file)

        except:
            print("Index for the failed files:", c)
            #introduction_vector.append([])
            #abstract_vector.append([])
            #author_names_vector.append([])
            # texts.append([])
            # filenames.append([])
            # folder_names.append([])

Index for the failed files: 1
Index for the failed files: 2
Index for the failed files: 3
Index for the failed files: 4
Index for the failed files: 5
<class 'dict'>
abstract_start 572
<class 'dict'>
abstract_start 552
Index for the failed files: 8
Index for the failed files: 9
Index for the failed files: 10
<class 'dict'>
abstract_start 704
Index for the failed files: 12
<class 'dict'>
abstract_start 468
Index for the failed files: 14
Index for the failed files: 15
Index for the failed files: 16
Index for the failed files: 17
Index for the failed files: 18
Index for the failed files: 19
Index for the failed files: 20
<class 'dict'>
abstract_start 792
Index for the failed files: 22
Index for the failed files: 23
<class 'dict'>
abstract_start 2295
<class 'dict'>
abstract_start 794
Index for the failed files: 26
Index for the failed files: 27
<class 'dict'>
abstract_start 828
Index for the failed files: 29
<class 'dict'>
abstract_start 499
Index for the failed files: 31
Index for the fail

Index for the failed files: 36
Index for the failed files: 37
Index for the failed files: 38
Index for the failed files: 39
<class 'dict'>
abstract_start 4170
<class 'dict'>
abstract_start 121
<class 'dict'>
abstract_start 1133
<class 'dict'>
abstract_start 1370
Index for the failed files: 44
<class 'dict'>
abstract_start 606
<class 'dict'>
abstract_start 394
Index for the failed files: 47
Index for the failed files: 48
<class 'dict'>
abstract_start 379
Index for the failed files: 50
<class 'dict'>
abstract_start -1
Index for the failed files: 51
Index for the failed files: 52
Index for the failed files: 53
Index for the failed files: 54
Index for the failed files: 55
Index for the failed files: 56
Index for the failed files: 57
Index for the failed files: 58
Index for the failed files: 59
Index for the failed files: 60
Index for the failed files: 61
Index for the failed files: 62
Index for the failed files: 63
Index for the failed files: 64
Index for the failed files: 65
Index for the

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(filenames,columns = ['FileName'])
df['Author'] = folder_names
df['Text'] = texts

In [ ]:
df.shape

(182, 3)

In [ ]:
df.head()

,FileName,Author,Text
0,Generalizing Surrogate-Assisted Evolutionary.pdf,Yew-Soon Ong,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2204.11835.pdf,Om Prakash Patel,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,OCC-90253_AM.pdf,Om Prakash Patel,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,1299-1309.pdf,Om Prakash Patel,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,bharill2018.pdf,Om Prakash Patel,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [ ]:
df.loc[0,'Text']

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nD:\\IEEE_FINAL\\22-4-2010\\forTEVC-paginatedpdf\\tevc-dlim-2027359\\tevc-dlim-2027359-proof\n\n\nIEEE TRANSACTIONS ON EVOLUTIONARY COMPUTATION, VOL. 14, NO. 3, JUNE 2010 329\n\nGeneralizing Surrogate-Assisted Evolutionary\nComputation\n\nDudy Lim, Yaochu Jin, Senior Member, IEEE, Yew-Soon Ong, Member, IEEE, and\nBernhard Sendhoff, Senior Member, IEEE\n\nAbstract—Using surrogate models in evolutionary search pro-\nvides an efficient means of handling today’s complex applica-\ntions plagued with increasing high-computational needs. Recent\nsurrogate-assisted evolutionary frameworks have relied on the\nuse of a variety of different modeling approaches to approximate\nthe complex problem landscape. From these recent studies, one\nmain research issue is with the choice of modeling scheme used,\nwhich has been found to affect the performance of evolutionary\nsearch significantly. Given that theoretical knowl

In [ ]:
df.to_csv('/content/drive/MyDrive/IK/IK_rr_DataFrame.csv', index = False)

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/IK'

In [ ]:
!ls /content/drive/MyDrive/IK/IK_rr_DataFrame.csv